In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import math
import pylab
from scipy.interpolate import interp1d
import time

In [ ]:
from driftcurve import driftcurve, single_time_driftcurve
import cr_data_inspection_functions as crd

# part 1 : load up the data

In [ ]:
fname = '/scr/kp/cr_data/overnight_scan_20-21April2023/overnight_software_snapshots1682051758.6163533.dat'
cr_events = crd.parsefile(fname, end_ind = 100)
cr_data_times = np.array([cr_events[i]['timestamp'] for i in range(len(cr_events))])
cr_data = np.array([cr_events[i]['data'] for i in range(len(cr_events))])

In [ ]:
ADC_to_V = 1.
P_data = (ADC_to_V * np.fft(cr_data))**2

# part 2 : set up the system of equations

In [ ]:
#gives skytemp in K and lst
#note that skytemp does not include antenna LNA gain
#lst, skytemp = driftcurve(do_plot=False)
dfreq = 10e6 # frequency steps
fmin =20e6
fmax = 81e6
freq = np.arange(fmin, fmax, dfreq)
Psky_at_time = []
for t in cr_data_times:
    Tsky = []
    for f in freq:
        lst, T = single_time_driftcurve(frequency = f)
        Tsky.append(T)
    Tsky = np.array(Tsky)
    k_b = 8.617333e-5 #V/K
    r_ant = 50. # ohms, this is a guess
    Psky = 2.0 * k_b * Tsky * dfreq * r_ant #in V^2
    Psky_at_time.append(Psky)
Psky_at_time = np.array(Psky_at_time).reshape(cr_data_times.shape[0], freq.shape[0])

In [ ]:
#need to load up antenna + LNA gain simulated
#this is what you need for Psim1 in lwa paper
#Psim1 = (Psky + N_lna) * G_ant*A_corr

def load_antenna_gain(freq):
    return np.ones(freq.shape, dtype=float)

G_ant = load_antenna_gain(freq)

In [ ]:
#next you need to load up simulated coax cable losses to get
#Psim2 = (Psky + N_lna) * G_ant*A_corr*L_coax

def load_coax_losses(freq):
    return np.ones(freq.shape, dtype=float)

L_coax = load_coax_losses(freq)

In [ ]:
#next you need to add in the last stage amplifier + ADC noise
#Psim3 = [(Psky + N_lna) * G_ant*A_corr*L_coax + N_RCU]*G_RCU*S_const + N_ADC

def load_RCU_gain(freq):
    return np.ones(freq.shape, dtype=float)

G_RCU = load_RCU_gain(freq)

# part 3 : use system of equations to find unknowns

In [ ]:
from scipy.optimize import curve_fit
def optfunc(x, N_lna, N_RCU, S_const, N_ADC, *A_corr):
    # G_ant, L_coax and G_RCU are implicit global here (dumb but okay for now)
    y = sum(((x_i + N_lna)*G_ant*A_i*L_coax + N_RCU)*G_RCU*S_const + N_ADC for x_i, A_i in zip(x, A_corr))
    return y

popt, pcov = curve_fit(optfunc, P_data, P_sky_at_time, p0 = np.ones(4 + freq.shape[0]))